### 环境配置
我们clone MetaGPT项目到本地进行研究：
```bash
!git clone https://github.com/geekan/MetaGPT.git
!cd MetaGPT
!pip install -e .
```

In [8]:
!git clone https://github.com/geekan/MetaGPT.git


Cloning into 'MetaGPT'...
remote: Enumerating objects: 32223, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 32223 (delta 20), reused 13 (delta 8), pack-reused 32182
Receiving objects: 100% (32223/32223), 92.49 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (24132/24132), done.


In [15]:
!cd ./MetaGPT/
!dir ./MetaGPT/

Dockerfile   README.md	  docs	    requirements.txt  tests
LICENSE      SECURITY.md  examples  ruff.toml
MANIFEST.in  config	  metagpt   setup.py


In [18]:
%pip install --upgrade -e ./MetaGPT/

Obtaining file:///workspaces/MetaGPT-Learn/MetaGPT
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached typer-0.9.0-py3-none-any.whl.metadata (14 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
Using cached typer-0.9.0-py3-none-any.whl (45 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Building editable for metagpt (pyproject.toml) ... done
  Created wheel for metagpt: filename=metagpt-0.8.1-0.editable-py3-none-any.whl size=9177 sha256=8c056108c85398c0646c5eb8e3171908b336cda7d67259075748d9b71ce66dc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_d3_k3/wheels/fc/8a/52/7a1c7d27adf6344504c52041d73fee26e3453f0630f1631db9
Successfully built metagpt
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uni

- 配置API Key,打开/MetaGPT/config/config2.yaml
- 然后填写自己的请求接口和API Key，作者这里使用的是deepseek的模型

In [1]:
# 测试一下接口
from openai import OpenAI

client = OpenAI(api_key="sk-6XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX74", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

# 运行正常没有问题
print(response.choices[0].message.content)

 Hello! How can I assist you today? If you have any questions or need information on a specific topic, feel free to ask.


###  编写CodeWrite动作


In [2]:
import re
import asyncio
from metagpt.actions import Action

class CodeWrite(Action):
    PROMPT_TEMPLATE: str = """
    根据以下需求，编写一个能够实现{requirements}的Python函数，并提供两个可运行的测试用例。
    返回的格式为：```python\n你的代码\n```，请不要包含其他的文本。
    ```python
    # your code here
    ```
    """

    name: str = "CodeWriter"

    async def run(self, requirements: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=requirements)
        rsp = await self._aask(prompt)
        code_text = CodeWrite.parse_code(rsp)
        return code_text

    @staticmethod
    def parse_code(rsp): # 从模型生成中字符串匹配提取生成的代码
        pattern = r'```python(.*?)```'  # 使用非贪婪匹配
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        # 添加注释
        code_lines = code_text.split('\n')
        commented_code_lines = [f'# {line}' if line.strip() == '' else line for line in code_lines]
        commented_code_text = '\n'.join(commented_code_lines)
        return commented_code_text

2024-05-15 18:11:36.284 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /workspaces/MetaGPT-Learn/MetaGPT


### 设计CodeWriterAgent

In [3]:
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

class CodeWriter(Role):
    """
    CodeWriter 角色类，继承自 Role 基类
    """

    name: str = "Cheems"  # 角色昵称
    profile: str = "CodeWriter"  # 角色人设

    def __init__(self, **kwargs):
        """
        初始化 CodeWriter 角色
        """
        super().__init__(**kwargs)  # 调用基类构造函数
        self._init_actions([CodeWrite])  # 为角色配备 CodeWrite 动作

    async def _act(self) -> Message:
        """
        定义角色行动逻辑
        """
        logger.info(f"{self._setting}: ready to {self.rc.todo}")  # 记录日志
        todo = self.rc.todo  # 获取待执行的动作 (CodeWriter)

        msg = self.get_memories(k=1)[0]  # 获取最近一条记忆 (用户输入)

        code_text = await todo.run(msg.content)  # 执行 CodeWrite 动作，获取生成的代码
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))  # 构造 Message 对象

        return msg  # 返回生成的 Message

### 运行测试CodeWriterAgent调用效果


In [7]:
import asyncio

async def main():
    msg = "如何用python获取最新的股票统计数据？"
    role = CodeWriter() # 实例化CodeWrite
    logger.info(msg) # 记录日志信息
    result = await role.run(msg) # 得到运行结果
    logger.info(result) # 记录运行结果

asyncio.run(main()) # 异步运行main函数 # 正常python文件使用

/home/codespace/.python/current/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: Cannot close a running event loop

In [1]:
import re
import asyncio
from metagpt.actions import Action
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

class CodeWrite(Action):
    PROMPT_TEMPLATE: str = """
    根据以下需求，编写一个能够实现{requirements}的Python函数，并提供两个可运行的测试用例。
    返回的格式为：```python\n你的代码\n```，请不要包含其他的文本。
    ```python
    # your code here
    ```
    """

    name: str = "CodeWriter"

    async def run(self, requirements: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=requirements)
        rsp = await self._aask(prompt)
        code_text = CodeWrite.parse_code(rsp)
        return code_text

    @staticmethod
    def parse_code(rsp): # 从模型生成中字符串匹配提取生成的代码
        pattern = r'```python(.*?)```'  # 使用非贪婪匹配
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        # 添加注释
        code_lines = code_text.split('\n')
        commented_code_lines = [f'# {line}' if line.strip() == '' else line for line in code_lines]
        commented_code_text = '\n'.join(commented_code_lines)
        return commented_code_text

class CodeWriter(Role):
    """
    CodeWriter 角色类，继承自 Role 基类
    """

    name: str = "Cheems"  # 角色昵称
    profile: str = "CodeWriter"  # 角色人设

    def __init__(self, **kwargs):
        """
        初始化 CodeWriter 角色
        """
        super().__init__(**kwargs)  # 调用基类构造函数
        self._init_actions([CodeWrite])  # 为角色配备 CodeWrite 动作

    async def _act(self) -> Message:
        """
        定义角色行动逻辑
        """
        logger.info(f"{self._setting}: ready to {self.rc.todo}")  # 记录日志
        todo = self.rc.todo  # 获取待执行的动作 (CodeWriter)

        msg = self.get_memories(k=1)[0]  # 获取最近一条记忆 (用户输入)

        code_text = await todo.run(msg.content)  # 执行 CodeWrite 动作，获取生成的代码
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))  # 构造 Message 对象

        return msg  # 返回生成的 Message

async def main():
    msg = "如何用python获取最新的股票统计数据？"
    role = CodeWriter() # 实例化CodeWrite
    logger.info(msg) # 记录日志信息
    result = await role.run(msg) # 得到运行结果
    logger.info(result) # 记录运行结果

asyncio.run(main()) # 异步运行main函数 # 正常python文件使用

2024-05-15 18:21:09.414 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /workspaces/MetaGPT-Learn/MetaGPT


RuntimeError: asyncio.run() cannot be called from a running event loop